In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install pmdarima

In [3]:
from benchmark import *
from pathlib import Path
from warnings import simplefilter
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import pmdarima
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import ADFTest

In [4]:
train_df = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")

In [5]:
qq= {0:'binance',1:'btc',2:'btc_cash',3:'cardano',4:'dodge',5:'eos',6:'eth',7:'eth_classic',8:'iota',9:'lite',10:'maker',11:'monero',12:'stellar',13:'tron'}

In [ ]:
#asset = {'binance':'0','btc':'1','2','3','4','5','6','7','8','9','10','11','12','13'}

In [100]:
#test_df = pd.read_csv("../input/g-research-crypto-forecasting/supplemental_train.csv")

In [6]:
binance = train_df[train_df["Asset_ID"]==0].reset_index(drop=True)
btc = train_df[train_df["Asset_ID"]==1].reset_index(drop=True) # Asset_ID = 1 for Bitcoin
btc_cash = train_df[train_df["Asset_ID"]==2].reset_index(drop=True)
cardano = train_df[train_df["Asset_ID"]==3].reset_index(drop=True)
dodge = train_df[train_df["Asset_ID"]==4].reset_index(drop=True)  
eos = train_df[train_df["Asset_ID"]==5].reset_index(drop=True)
eth = train_df[train_df["Asset_ID"]==6].reset_index(drop=True)
eth_classic = train_df[train_df["Asset_ID"]==7].reset_index(drop=True)
iota = train_df[train_df["Asset_ID"]==8].reset_index(drop=True)
lite = train_df[train_df["Asset_ID"]==9].reset_index(drop=True)
maker = train_df[train_df["Asset_ID"]==10].reset_index(drop=True)
monero = train_df[train_df["Asset_ID"]==11].reset_index(drop=True)
stellar = train_df[train_df["Asset_ID"]==12].reset_index(drop=True)
tron = train_df[train_df["Asset_ID"]==13].reset_index(drop=True)

In [7]:
listOfDataframe =[binance, btc, btc_cash, cardano, dodge, eos, eth, eth_classic, iota, lite, maker, monero, stellar, tron]

In [8]:
details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

In [9]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_ID)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = (
            price.shift(periods=-16) /
            price.shift(periods=-1)
        ) - 1
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)
    
    return targets

In [10]:
def Predict(asset,timestamp,row):
    y = pd.DataFrame(columns = ['timestamp','Asset_ID','Close'])
    listOfDataframe[asset] = listOfDataframe[asset].append(row)
    listOfDataframe[asset]=listOfDataframe[asset].sort_values(by= 'timestamp',ascending=True)
    Train = listOfDataframe[asset][(listOfDataframe[asset].timestamp<=timestamp)][-2000:]
    train_model = Train['Close'].values
    history = [x for x in train_model]
    model = ARIMA(history, order=(1, 1, 3))
    fitted = model.fit()
    output = fitted.forecast(17, alpha=0.05)
    for t in range(17):
        bb = {'timestamp':timestamp+60*t,'Asset_ID':asset,'Close':output[t]}
        y = y.append(bb,ignore_index=True)
    y[['timestamp', 'Asset_ID']] = y[['timestamp', 'Asset_ID']].astype(int)
    return y

In [11]:
def findPrice(asset,timestamp,row):
    asset = asset
    dataset = listOfDataframe[asset]
    TargetPrice = dataset[(dataset.timestamp<=timestamp+960)&(dataset.timestamp>=timestamp+60)]
    #Target16 = dataset[dataset['timestamp']==timestamp+960]
    #Target1 = dataset[dataset['timestamp']==timestamp+60]
    timestamp = timestamp
    row = row
    if TargetPrice.empty:
        output = Predict(asset,timestamp,row)
        return output
    else:
        return TargetPrice

In [12]:
ids = list(details.Asset_ID)

In [13]:
def Forecast(test_df):
    x = pd.DataFrame(columns = ['timestamp','Asset_ID','Close'])
    for i in range(len(test_df)):
        row = test_df[test_df.index==i]
        asset = row['Asset_ID'].values[0]
        timestamp = row['timestamp'].values[0]
        #listOfDataframe[asset]=listOfDataframe[asset].append(row, ignore_index=True)
        TargetPrice = findPrice(asset,timestamp,row)
        x = x.append(TargetPrice,ignore_index=True)
    timestamp = test_df['timestamp'].values[0]
    t = train_df[train_df.timestamp<=timestamp]
    t = t[-100000:]
    t = t.drop(columns = ['Count','Open','High','Low','Volume','VWAP','Target'])
    t = t.append(x,ignore_index=True)
    price_column = 'Close'
    t['Time'] = pd.to_datetime(t['timestamp'], unit='s')
    k = calculate_target(t, details, price_column)
    target = k[k.index==timestamp]
    return target

In [14]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [15]:
for (test_df, sample_prediction_df) in iter_test:
    Target = Forecast(test_df)
    Target = Target.values
    outcome = pd.DataFrame()
    order = test_df['Asset_ID'].values
    for k in order:
        line = ids.index(k)
        outcome=outcome.append({'Target':Target[0][line]},ignore_index=True)
    sample_prediction_df['Target'] = outcome['Target']
    env.predict(sample_prediction_df)